# collect all data and put them in a single dataframe

In [ ]:
import os
import pandas as pd

checked_path = "/home/boonstra/24_04_11_with_checks"
unchecked_path = "/home/boonstra/24_04_11_without_checks"

def parse_data(data_path):
    data_frames = []
    for filename in os.listdir(checked_path):
        seed, extension = os.path.splitext(filename)
        if extension.lower() != ".csv":
            continue
        filepath = os.path.join(data_path, filename)
        try:
            df = pd.read_csv(filepath, header=0)
        except FileNotFoundError:
            continue
        df["seed"] = seed
        data_frames.append(df)

    all_data = pd.concat(data_frames)
    all_data.index = range(len(all_data))
    return all_data

checked_data = parse_data(checked_path)
unchecked_data = parse_data(unchecked_path)

checked_final_task_data = checked_data[checked_data["task"] == "C8"]
unchecked_final_task_data = unchecked_data[unchecked_data["task"] == "C8"]

In [ ]:
import numpy as np
print(len(checked_data[checked_data["scenario"].notna()]))
print(len(checked_data[np.logical_and(checked_data["scenario"].notna(), checked_data["error_corrected"] != True)]))
print(len(unchecked_data[unchecked_data["scenario"].notna()]))
print(len(unchecked_data[np.logical_and(unchecked_data["scenario"].notna(), unchecked_data["error_corrected"] != True)]))

In [ ]:
import numpy as np
def filter_error_included_simulations(data):
    error_included_data = []
    for _, data in data.groupby("seed"):
        if any(np.logical_and(data["scenario"].notna(), data["error_corrected"] != True)):

            error_included_data.append(data)
    error_included_data = pd.concat(error_included_data)

    return error_included_data

checked_error_included_data = filter_error_included_simulations(checked_data)
unchecked_error_included_data = filter_error_included_simulations(unchecked_data)

checked_final_task_error_included_data = checked_error_included_data[checked_error_included_data["task"] == "C8"]
unchecked_final_task_error_included_data = unchecked_error_included_data[unchecked_error_included_data["task"] == "C8"]


In [ ]:
import numpy as np

def obtain_log_values(data):
    return [np.log10(v) for v in data["total"] if v >= 2e-7]

checked_log_values = obtain_log_values(checked_final_task_data)
checked_error_included_log_values = obtain_log_values(checked_final_task_error_included_data)
unchecked_log_values = obtain_log_values(unchecked_final_task_data)
unchecked_error_included_log_values = obtain_log_values(unchecked_final_task_error_included_data)

In [ ]:
from matplotlib.axes._axes import Axes
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import laplace

mean=0.000161
fig, axs = plt.subplots(1,2)
scale_factor = 3
n_bins=50
fig.set_size_inches(w=scale_factor*80/25.4, h=scale_factor*40/25.4)

for data_label, y_pos, color in zip(["unchecked", "checked"], [0.95,0.9], ["C0", "C1"]):
    x_pos = 0.02 if data_label == "checked" else 0.5
    data = globals()[f"{data_label}_final_task_data"]
    log_values = globals()[f"{data_label}_log_values"]
    n_values = len(data)
    n_failure_free = n_values - len(globals()[f"{data_label}_final_task_error_included_data"])
    # print(len(globals()[f"{data_label}_final_task_error_included_data"]))
    n_zero_values = n_values - len(log_values)


    lap_loc = np.log10(mean)
    lap_pairs = sorted([(abs(v - lap_loc), v) for v in log_values], key=lambda x: x[0])
    lap_pairs = lap_pairs[n_failure_free:]
    lap_differences, lap_values = zip(*lap_pairs)
    lap_scale = np.mean(lap_differences)

    ax: Axes = axs[0]
    n, bins, patches = ax.hist(log_values, bins=n_bins, linewidth=0.5, edgecolor="black")
    bin_width = bins[1] - bins[0]
    lap_loc, lap_scale = laplace.fit(lap_values)
    lap_x = np.arange(-7, 0.01, 0.01)
    lap_y = laplace.pdf(lap_x, loc=lap_loc, scale=lap_scale) * len(lap_values) * bin_width
    # ax.plot(lap_x, lap_y, color="red", linewidth=1.0)
    ax.set_title("all results")
    ax.set_xlabel("Failure probability [-]")
    ax.set_ylabel("Frequency [%]")
    axs[0].text(x=x_pos, y= 0.95, s=f"{data_label}",transform=ax.transAxes, weight="bold", color=color)
    axs[0].text(x=x_pos, y= 0.90, s=f"{data_label} peak: {round(max(n)/1000,2)}%",transform=ax.transAxes, color=color)
    axs[0].text(x=x_pos, y= 0.85, s=f"zero-probability: {round(n_zero_values/1000,1)}%",transform=ax.transAxes, color=color)
    axs[0].text(x=x_pos, y= 0.8, s=f"error-free: {round(n_failure_free/1000,1)}%",transform=ax.transAxes, color=color)
    axs[0] = ax

    # plot zoomed in figure on degradations
    ax: Axes = axs[1]
    n, bins, patches = ax.hist(log_values, bins=bins, linewidth=0.5, edgecolor="black")
    # ax.plot(lap_x, lap_y, color="red", linewidth=1.0)
    ax.set_title("degradations")
    ax.set_xlabel("Failure probability [-]")
    ax.set_ylabel("")
    axs[1] = ax


y_lims = [5,2.0]
y_tick_increment=[1.0,0.5]
x_lims = [-7, -3.5]
for i in [0,1]:
    ax: Axes = axs[i]
    if y_lims[i] is None:
        _, y_max = ax.get_ylim()
    else:
        y_max = y_lims[i] * n_values / 100
    x_min = x_lims[i]
    ax.set_ylim(0,y_max)
    ax.set_xlim(x_min, 0)
    axs[i] = ax

fig.suptitle("Failure probabilities after simulations")
# fig.tight_layout()
for i in [0,1]:
    ax: Axes = axs[i]

    if y_lims[i] is None:
        _, y_max = ax.get_ylim()
        y_max /= n_values/100
    else:
        y_max = y_lims[i]
    x_min = x_lims[i]

    
    x_tick_positions = ax.get_xticks()
    x_tick_labels = ["$\mathregular{10^{%s}}$" %(item,) for item in x_tick_positions]
    ax.set_xticks(x_tick_positions, x_tick_labels)

    y_tick_values = np.arange(0, y_max + y_tick_increment[i], y_tick_increment[i])
    y_tick_positions = y_tick_values * n_values / 100
    y_tick_labels = [f"{round(item,1)}" for item in y_tick_values]
    ax.set_yticks(y_tick_positions, y_tick_labels)
    axs[i] = ax
plt.tight_layout()
fig.savefig(f"probabilities_after_simulations.png")


In [ ]:
print(len(checked_final_task_error_included_data[checked_final_task_error_included_data["total"] > 0.00016143])/n_values)
print(len(checked_final_task_error_included_data[checked_final_task_error_included_data["total"] < 0.00016143])/n_values)